In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive/exercise

/content/drive/MyDrive/exercise


# Import Module

In [ ]:
import pandas as pd
import numpy as np
import random
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Prepare Data

## Define Function

In [ ]:
def bmi_calc(x):
    return np.round(x['ITEM_F002'] / ((x['ITEM_F001']/100)**2))

In [ ]:
def age_categorize(x):
    if x.TEST_AGE < 10:
        return '10세 미만'
    elif x.TEST_AGE >= 70:
        return '70대 이상'
    else:
        return str((x.TEST_AGE // 10) * 10) + '대'
     

In [ ]:
def bmi_categorize(x):
    bmi = bmi_calc(x)
    return '3단계비만' if bmi>=35 else ('2단계비만' if bmi>=30 else \
                                  ('1단계비만' if bmi>=25 else \
                                   ('비만전단계' if bmi>=23 else \
                                    ('정상체중' if bmi>=18.5 else '저체중'))))

## Load Data

In [ ]:
data = pd.read_csv('./processing/processed_data.csv')
user_group = pd.read_csv('./processing/user_group.csv')
sample_data = pd.read_csv('./test_data.csv')

In [ ]:
data.head(3)

,USER_ID,MESURE_AGE_CO,COAW_FLAG_NM,MESURE_DAY,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,PRE_EX,MAIN_EX,FINISH_EX,MESURE_IEM_018_VALUE,AGE_GROUP,BMI_GROUP,CERT_GROUP,G_ID,TEST_SEX
0,0,23,2등급,20220502,F,159.2,56.2,정적 스트레칭 루틴프로그램,"전완대고 버티기,윗몸 일으키기,앉아서 다리 밀기,앉아서 다리 펴기,뒤꿈치 들기,누워...","걷기,정적 스트레칭 루틴프로그램",22.2,20대,정상체중,중,79,M
1,1,23,2등급,20220502,F,159.2,56.2,정적 스트레칭 루틴프로그램,"전완대고 버티기,윗몸 일으키기,앉아서 다리 밀기,앉아서 다리 펴기,뒤꿈치 들기,누워...","걷기,정적 스트레칭 루틴프로그램",22.2,20대,정상체중,중,82,F
2,2,23,참가증,20220502,F,161.9,62.7,정적 스트레칭 루틴프로그램,"전완대고 버티기,윗몸 일으키기,앉아서 다리 밀기,앉아서 다리 펴기,뒤꿈치 들기,누워...","걷기,정적 스트레칭 루틴프로그램",23.9,20대,비만전단계,하,86,M


In [ ]:
user_group.head(3)

,G_ID,AGE_GROUP,BMI_GROUP,TEST_SEX,CERT_GROUP
0,0,10세 미만,저체중,M,상
1,1,10세 미만,저체중,M,중
2,2,10세 미만,저체중,M,하


In [ ]:
user_group["G_ID"].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
sample_data.head(3)

,TEST_AGE,CERT_GBN,TEST_SEX,ITEM_F001,ITEM_F002
0,25,하,M,167.3,46.98
1,19,하,M,179.0,84.00
2,19,하,M,171.5,95.60


## prepare smaple user data

In [ ]:
sample_data = sample_data.iloc[:300].rename(columns={'CERT_GBN':'CERT_GROUP'})
sample_data['ITEM_F018'] = sample_data.apply(bmi_calc, axis=1)
sample_data['AGE_GROUP'] = sample_data.apply(age_categorize,axis=1)
sample_data['BMI_GROUP'] = sample_data.apply(bmi_categorize,axis=1)
sample_data.head(5)

,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP
0,25,하,M,167.3,46.98,17.0,20대,저체중
1,19,하,M,179.0,84.00,26.0,10대,1단계비만
2,19,하,M,171.5,95.60,33.0,10대,2단계비만
3,20,하,F,158.4,43.84,17.0,20대,저체중
4,53,하,M,173.4,73.40,24.0,50대,비만전단계


In [ ]:
group_sample_data = pd.merge(sample_data,user_group,how='left').reset_index().rename(columns={'index':'USER_ID'})
group_sample_data.head(5)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID
0,0,25,하,M,167.3,46.98,17.0,20대,저체중,74
1,1,19,하,M,179.0,84.00,26.0,10대,1단계비만,56
2,2,19,하,M,171.5,95.60,33.0,10대,2단계비만,62
3,3,20,하,F,158.4,43.84,17.0,20대,저체중,77
4,4,53,하,M,173.4,73.40,24.0,50대,비만전단계,194


# Exercise Recommendation

## Count Exercise Set

In [ ]:
set_df = data[['G_ID','PRE_EX','MAIN_EX','FINISH_EX']].value_counts().reset_index().rename(columns={0:'COUNT'})

## Define function

In [ ]:

# 운동 리스트를 통해 희소행렬 도출
# input : exercise list / output : sparse matrix
def sparse_matrix(ex_list):
  te = TransactionEncoder()
  te_ary = te.fit(ex_list).transform(ex_list)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_)
  return sparse_matrix

In [ ]:
# 연관분석을 통해 선택된 운동과 연관성이 높은 운동 3개를 리턴
def ranking(pre_df, main_df, finish_df, ex_list):
  global df_list
  df_list = [pre_df, main_df, finish_df] # 단계별 연관분석 결과값
  print("df_list",main_df.columns)
  result_list = [] # 결과값을 리스트에 담아 하나씩 출력
  result_df = pd.DataFrame(columns=main_df.columns) # 빈 DataFrame 생성

  for i in range(len(df_list)): # 운동 단계별 for문 pre, main, finish
    global tmp_df 
    tmp_df = df_list[i] 
    for each_ex in ex_list[i].split(','): # 각 단계별 운동들을 리스트로 분할
      each_ex = each_ex # 운동이름에 공백 제거
      if tmp_df.empty:
        result_df = pd.DataFrame(columns=main_df.columns)
        print("empty result_df",result_df)
      else:
        each_result = tmp_df[tmp_df['antecedents']==frozenset({each_ex})].sort_values('support') # 추천 세트 운동과의 연관분석 결과를 지지도 순으로 정렬
        each_result['consequents'] = each_result['consequents'].apply(lambda x : list(x)[0]) # frozenset to string, 타입변환
        print("each_result", each_result)
        length = 3 if len(each_result) else len(each_result) # set length
        result_df = pd.concat([result_df,each_result.iloc[:length]]) # 연관성이 높은 운동이 3개 이상일 경우 3개만 저장
    print("cosequents",result_df["consequents"])
    result_list.append(result_df['consequents']) # 결과값 리스트에 추가
    result_df = None # 초기화
  return list(result_list[0]), list(result_list[1]), list(result_list[2]) # 준비, 본, 마무리 운동

In [ ]:

# select main routine and additional routine
def select_exercise(pre_list, main_list, finish_list, max):
  main_reco_list = [] # 메인 추천 리스트
  sub_reco_list = [] # 추가 추천 리스트
  tmp_list = []
  asso_list = [pre_list,main_list,finish_list] # 연관 있는 운동 리스트
  tmp_set = set()
  random_num = random.randint(1, 100) # 임의의 1개 운동을 선택하기 위한 난수 생성
  
  for i, str_ in enumerate(max):
    ex_list = str_.split(',') # 운동 세트에서 각 단계별 운동들
    rand_num = random_num%len(ex_list)
    reco_ex = ex_list[rand_num] # 난수를 통해 임의의 1개 운동 선택
    main_reco_list.append(reco_ex) # 메인 추천 리스트에 선택된 운동 삽입

    tmp_set.update(ex_list) # 임시 리스트에 max 운동 리스트 삽입
    tmp_set.update(asso_list[i]) # 연관 분석을 통해 추천된 운동들 추가
    tmp_set.remove(reco_ex) # 선택된 운동 삭제 -> 단계별 최빈 세트 운동에서 메인 추천 운동 제거
    tmp_list.append(list(tmp_set)) # 해당 set을 리스트로 변환하여 tmp_list에 추가 -> [ [준,], [본,], [마,] ]
  for i, reco in enumerate(tmp_list):
    if (len(reco)) == 0:
      random_list = ['없음']
    elif (len(reco)) < 4:
      random_list = reco # 운동 개수가 4개 이하면 전체 운동 적용
    else:
      random_list = random.sample(reco, 4) # 운동 개수가 4개 초과일 경우 임의로 4개 선택
    sub_reco_list.append(random_list)
  return list(main_reco_list), list(sub_reco_list)
     

In [ ]:


# select top 1 exercise set from user info
# input : group_id / ouput : recommend_set, sub_recommend_set
def mode_set(g_id, min_support = 0.01, min_threshold=0.2):
  print("## g_id: ",g_id)
  user_set = set_df[set_df['G_ID']==g_id] # 유저가 해당하는 그룹의 처방 운동 세트 빈도
  #print(list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]))
  max_ex_set = list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]) # 가장 많이 처방된 운동 세트
  #print(max_ex_set)


  # 해당 그룹의 단계별 운동 리스트 생성
  pre_ex_all = list(user_set['PRE_EX'].apply(lambda x : x.split(','))) 
  main_ex_all = list(user_set['MAIN_EX'].apply(lambda x : x.split(',')))
  finish_ex_all = list(user_set['FINISH_EX'].apply(lambda x : x.split(',')))

  # 단계별 sparse matrix 생성
  pre_sparse = sparse_matrix(pre_ex_all)
  main_sparse = sparse_matrix(main_ex_all)
  finish_sparse = sparse_matrix(finish_ex_all)

  # 지지도가 0.01 이상인 운동 선택
  pre_itemsets = apriori(pre_sparse, min_support = min_support, use_colnames=True,max_len=2)
  main_itemsets = apriori(main_sparse, min_support = min_support, use_colnames=True,max_len=2)
  finish_itemsets = apriori(finish_sparse, min_support = min_support, use_colnames=True,max_len=2)

  # 그 중에서 신뢰도가 0.2 이상인 운동 선택 (원본에서는 0.2임)
  pre_select = association_rules(pre_itemsets, metric="confidence", min_threshold=min_threshold)
  main_select = association_rules(main_itemsets, metric="confidence", min_threshold=min_threshold)
  finish_select = association_rules(finish_itemsets, metric="confidence", min_threshold=min_threshold)
  print("pre_select",pre_select)

  # consequents의 길이가 1인 것만 선택 -> 1개의 운동끼리 비교해서 추천하도록
  pre_select = pre_select[pre_select['consequents'].apply(lambda x : len(x) == 1)]
  main_select = main_select[main_select['consequents'].apply(lambda x : len(x) == 1)] 
  finish_select = finish_select[finish_select['consequents'].apply(lambda x : len(x) == 1)]
  print("pre_select after", pre_select)
  # ranking 
  pre_rank, main_rank, finish_rank = ranking(pre_select, main_select, finish_select, max_ex_set)

  # select exercise 
  recommend_set, sub_recommend_set = select_exercise(pre_rank, main_rank, finish_rank, max_ex_set)
  return recommend_set, sub_recommend_set

## Recommendation

In [ ]:
main_reco = []
sub_reco = []
u_id_list = []

In [ ]:

for u_id, g_id in group_sample_data[['USER_ID','G_ID']].values:
  if g_id in data["G_ID"].unique():
    best, sub = mode_set(g_id) # 추천 루틴 및 추가 추천 운동
    main_reco.append(list(best)) 
    sub_reco.append(sub)
    u_id_list.append(u_id)
     

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
62      (계단 뛰기)                  뒤꿈치 들기            0.051282   
67      (계단 뛰기)          매달려서 뒤로 팔굽혀 펴기            0.051282   
132     (계단 뛰기)             허리 굽혀 덤벨 들기            0.051282   
117     (계단 뛰기)                   윗몸올리기            0.051282   
89      (계단 뛰기)                      수영            0.051282   

     consequent support   support  confidence       lift  leverage  conviction  
7              0.179487  0.025641         0.5   2.785714  0.016437    1.641026  
88             0.025641  0.025641         0.5  19.500000  0.024326    1.948718  
91             0.025641  0.025641         0.5  19.500000  0.024326    1.948718  
93             0.025641  0.025641         0.5  19.500000  0.024326    1.948718  
95             0.025641  0.025641         0.5  19.500000  0.024326    1.948718  
97             0.051282  0.025641         0.5   9.750000  0.023011    1.897436  
99             0.025641  0.025641         0.5  19.500000  0.024326    1.948718  


In [ ]:
est, sub = mode_set(80) 

## g_id:  80
pre_select                antecedents      consequents  antecedent support  \
0          (가슴/어깨 앞쪽 스트레칭)   (등/어깨 뒤쪽 스트레칭)            0.038760   
1           (등/어깨 뒤쪽 스트레칭)  (가슴/어깨 앞쪽 스트레칭)            0.062016   
2                 (목 스트레칭)  (가슴/어깨 앞쪽 스트레칭)            0.062016   
3          (가슴/어깨 앞쪽 스트레칭)         (목 스트레칭)            0.038760   
4          (가슴/어깨 앞쪽 스트레칭)      (아래 팔 스트레칭)            0.038760   
..                     ...              ...                 ...   
68  (좌식생활자를 위한 동적 루틴 스트레칭)     (전신 루틴 스트레칭)            0.046512   
69            (하지 루틴 스트레칭)     (전신 루틴 스트레칭)            0.116279   
70               (허리 스트레칭)     (전신 루틴 스트레칭)            0.046512   
71              (척추 들어올리기)        (허리 스트레칭)            0.023256   
72               (허리 스트레칭)       (척추 들어올리기)            0.046512   

    consequent support   support  confidence       lift  leverage  conviction  
0             0.062016  0.038760    1.000000  16.125000  0.036356         inf  
1          

In [ ]:
best, sub = mode_set(253) 

## g_id:  253
pre_select Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []
pre_select after Empty DataFrame
Columns: []
Index: []
df_list Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')
empty result_df Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []
cosequents Series([], Name: consequents, dtype: object)
each_result    antecedents    consequents  antecedent support  consequent support  \
1   (넙다리 스트레칭)  가슴/어깨 앞쪽 스트레칭                 1.0                 1.0   
12  (넙다리 스트레칭)   등/어깨 뒤쪽 스트레칭                 1.0                 1.0   
15  (넙다리 스트레칭)         목 스트레칭                 1.0                 1.0   
16  (넙다리 스트레칭)         배 스트레칭                 1.0        

In [ ]:
# DataFrame 생성
reco_df = pd.DataFrame(main_reco,index=u_id_list,columns=['Recommend_pre_ex','Recommend_main_ex','Recommend_finish_ex'])
sub_df = pd.DataFrame(sub_reco,index=u_id_list,columns=['Additional_pre_ex','Additional_main_ex','Additional_finish_ex'])
merge_df = pd.merge(reco_df,sub_df,left_index=True,right_index=True).reset_index().rename(columns={'index':'USER_ID'})
result_df = pd.merge(group_sample_data,merge_df)
for column in result_df.iloc[:,-3:]:
  result_df[column] = result_df[column].apply(lambda x : (',').join(x))
     

In [ ]:
result_df.head(3)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID,Recommend_pre_ex,Recommend_main_ex,Recommend_finish_ex,Additional_pre_ex,Additional_main_ex,Additional_finish_ex
0,0,25,하,M,167.3,46.98,17.0,20대,저체중,74,유산소 운동 전 동적 루틴 스트레칭,윗몸올리기,전신 루틴 스트레칭,자가근막이완술 루틴 스트레칭,"엎드려서 팔 다리 들기,줄넘기,누워서 엉덩이 들어올리기,한발 앞으로 내밀고 앉았다 ...","네발기기 자세로 팔 다리 들기,엎드려 버티기,누워서 다리 들어올리기,누워서 하늘 자전거"
1,1,19,하,M,179.0,84.00,26.0,10대,1단계비만,56,옆으로 걷기,버피운동,등/어깨 뒤쪽 스트레칭,"맨몸운동 루틴프로그램,전사자세,네발기기 자세로 팔 다리 들기","가슴/어깨 앞쪽 스트레칭,팔벌려뛰기,넙다리 스트레칭,팔굽혀 펴기","뒤로 팔굽혀펴기,전사자세,배 스트레칭,팔굽혀펴기"
2,2,19,하,M,171.5,95.60,33.0,10대,2단계비만,62,네발기기 자세로 팔 다리 들기,소파를 이용한 윗몸올리기,어깨 뒤쪽 스트레칭,"맨몸운동 루틴프로그램,전사자세,옆으로 걷기,동적 스트레칭 루틴프로그램","허리 굽혀 덤벨 들기,스텝퍼 올라가서 점프하여 착지하기,앉아서 다리 밀기,목 스트레칭","수영,앉아서 다리 밀기,팔벌려뛰기,목 스트레칭"


In [ ]:
result_df.to_csv('../Data/Sample/sample_recommend_result.csv',encoding='utf-8',index=False)

OSError: ignored